# Tutorial A5 (Breaking AES-256 Bootloader)

This tutorial will take you through a complete attack on an encrypted bootloader using AES-256. This demonstrates how to use side-channel power analysis on practical systems, along with discussing how to perform analysis with different Analyzer models.

## Background

In the world of microcontrollers, a bootloader is a special piece of firmware that is made to let the user upload new programs into memory. This is especially useful for devices with complex code that may need to be patched or otherwise updated in the future - a bootloader makes it possible for the user to upload a patched version of the firmware onto the micro. The bootloader receives information from a communication line (a USB port, serial port, ethernet port, WiFi connection, etc...) and stores this data into program memory. Once the full firmware has been received, the micro can happily run its updated code.

There is one big security issue to worry about with bootloaders. A company may want to stop their customers from writing their own firmware and uploading it onto the micro. For example, this might be for protection reasons - hackers might be able to access parts of the device that weren't meant to be accessed. One way of stopping this is to add encryption. The company can add their own secret signature to the firmware code and encrypt it with a secret key. Then, the bootloader can decrypt the incoming firmware and confirm that the incoming firmware is correctly signed. Users will not know the secret key or the signature tied to the firmware, so they won't be able to "fake" their own.

This tutorial will work with a simple AES-256 bootloader. The victim will receive data through a serial connection, decrypt the command, and confirm that the included signature is correct. Then, it will only save the code into memory if the signature check succeeded. To make this system more robust against attacks, the bootloader will use cipher-block chaining (CBC mode). Our goal is to find the secret key and the CBC initialization vector so that we could successfully fake our own firmware.

### Bootloader Communications Protocol

The bootloader's communications protocol operates over a serial port at 38400 baud rate. The bootloader is always waiting for new data to be sent in this example; in real life one would typically force the bootloader to enter through a command sequence.

Commands sent to the bootloader look as follows:

```
       |<-------- Encrypted block (16 bytes) ---------->|
       |                                                |
+------+------+------+------+------+------+ .... +------+------+------+
| 0x00 |    Signature (4 Bytes)    |  Data (12 Bytes)   |   CRC-16    |
+------+------+------+------+------+------+ .... +------+------+------+
```

This frame has four parts:

* `0x00`: 1 byte of fixed header
* Signature: A secret 4 byte constant. The bootloader will confirm that this signature is correct after decrypting the frame.
* Data: 12 bytes of the incoming firmware. This system forces us to send the code 12 bytes at a time; more complete bootloaders may allow longer variable-length frames.
* CRC-16: A 16-bit checksum using the CRC-CCITT polynomial (0x1021). The LSB of the CRC is sent first, followed by the MSB. The bootloader will reply over the serial port, describing whether or not this CRC check was valid.

As described in the diagram, the 16 byte block is not sent as plaintext. Instead, it is encrypted using AES-256 in CBC mode. This encryption method will be described in the next section.

The bootloader responds to each command with a single byte indicating if the CRC-16 was OK or not:

```
            +------+
CRC-OK:     | 0xA1 |
            +------+

            +------+
CRC Failed: | 0xA4 |
            +------+
```
Then, after replying to the command, the bootloader veries that the signature is correct. If it matches the expected manufacturer's signature, the 12 bytes of data will be written to flash memory. Otherwise, the data is discarded.

### Details of AES-256 CBC

The system uses the AES algorithm in Cipher Block Chaining (CBC) mode. In general one avoids using encryption 'as-is' (i.e. Electronic Code Book), since it means any piece of plaintext always maps to the same piece of ciphertext. Cipher Block Chaining ensures that if you encrypted the same thing a bunch of times it would always encrypt to a new piece of ciphertext.

You can see another reference on the design of the encryption side; we'll be only talking about the decryption side here. In this case AES-256 CBC mode is used as follows, where the details of the AES-256 Decryption block will be discussed in detail later:

![AES-256](https://wiki.newae.com/images/8/88/Aes256_cbc.png)

This diagram shows that the output of the decryption is no longer used directly as the plaintext. Instead, the output is XORed with a 16 byte mask, which is usually taken from the previous ciphertext. Also, the first decryption block has no previous ciphertext to use, so a secret initialization vector (IV) is used instead. If we are going to decrypt the entire ciphertext (including block 0) or correctly generate our own ciphertext, we'll need to find this IV along with the AES key.

### Attacking AES-256

The system in this tutorial uses AES-256 encryption, which has a 256 bit (32 byte) key - twice as large as the 16 byte key we've attacked in previous tutorials. This means that our regular AES-128 CPA attacks won't quite work. However, extending these attacks to AES-256 is fairly straightforward: the theory is explained in detail in Extending AES-128 Attacks to AES-256.

As the theory page explains, our AES-256 attack will have 4 steps:

1. Perform a standard attack (as in AES-128 decryption) to determine the first 16 bytes of the key, corresponding to the 14th round encryption key.
1. Using the known 14th round key, calculate the hypothetical outputs of each S-Box from the 13th round using the ciphertext processed by the 14th round, and determine the 16 bytes of the 13th round key manipulated by inverse MixColumns.
1. Perform the MixColumns and ShiftRows operation on the hypothetical key determined above, recovering the 13th round key.
1. Using the AES-256 key schedule, reverse the 13th and 14th round keys to determine the original AES-256 encryption key.

## Firmware

For this tutorial, we'll be using the `bootloader-aes256` project, which we'll build as usual:

In [36]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET="NONE"

In [37]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/bootloader-aes256
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- bootloader-aes256-CWLITEARM.hex
rm -f -- bootloader-aes256-CWLITEARM.eep
rm -f -- bootloader-aes256-CWLITEARM.cof
rm -f -- bootloader-aes256-CWLITEARM.elf
rm -f -- bootloader-aes256-CWLITEARM.map
rm -f -- bootloader-aes256-CWLITEARM.sym
rm -f -- bootloader-aes256-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- bootloader.s aes256.s crcccitt.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- bootloader.d aes256.d crcccitt.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- bootloader.i aes256.i crcccitt.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (15:6.3.1+svn253039-1build1) 6.3.1 20170620
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: bootloader.c
arm-none-eabi-gcc -c

## Capturing Traces

### Setup

To start, we'll proceed with setup as usual:

In [1]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [2]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [7]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/bootloader-aes256/bootloader-aes256-CWLITEARM.hex"
#%run "Helper Scripts/Program_XMEGA.ipynb"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"

In [38]:
program_target(scope, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 5919 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5919 bytes


### Calculating the CRC

The next step we'll need to take in attacking this target is to communicate with it. Most of the transmission is fairly straight forward, but the CRC is a little tricky. Luckily, there's a lot of open source out there for calculating CRCs. In this case, we'll pull some code from pycrc:

In [3]:
# Class Crc
#############################################################
# These CRC routines are copy-pasted from pycrc, which are:
# Copyright (c) 2006-2013 Thomas Pircher <tehpeh@gmx.net>
#
class Crc(object):
    """
    A base class for CRC routines.
    """

    def __init__(self, width, poly):
        """The Crc constructor.

        The parameters are as follows:
            width
            poly
            reflect_in
            xor_in
            reflect_out
            xor_out
        """
        self.Width = width
        self.Poly = poly


        self.MSB_Mask = 0x1 << (self.Width - 1)
        self.Mask = ((self.MSB_Mask - 1) << 1) | 1

        self.XorIn = 0x0000
        self.XorOut = 0x0000

        self.DirectInit = self.XorIn
        self.NonDirectInit = self.__get_nondirect_init(self.XorIn)
        if self.Width < 8:
            self.CrcShift = 8 - self.Width
        else:
            self.CrcShift = 0

    def __get_nondirect_init(self, init):
        """
        return the non-direct init if the direct algorithm has been selected.
        """
        crc = init
        for i in range(self.Width):
            bit = crc & 0x01
            if bit:
                crc ^= self.Poly
            crc >>= 1
            if bit:
                crc |= self.MSB_Mask
        return crc & self.Mask


    def bit_by_bit(self, in_data):
        """
        Classic simple and slow CRC implementation.  This function iterates bit
        by bit over the augmented input message and returns the calculated CRC
        value at the end.
        """
        # If the input data is a string, convert to bytes.
        if isinstance(in_data, str):
            in_data = [ord(c) for c in in_data]

        register = self.NonDirectInit
        for octet in in_data:
            for i in range(8):
                topbit = register & self.MSB_Mask
                register = ((register << 1) & self.Mask) | ((octet >> (7 - i)) & 0x01)
                if topbit:
                    register ^= self.Poly

        for i in range(self.Width):
            topbit = register & self.MSB_Mask
            register = ((register << 1) & self.Mask)
            if topbit:
                register ^= self.Poly

        return register ^ self.XorOut
    
bl_crc = Crc(width = 16, poly=0x1021)

Now we can easily get the CRC for our message by calling `bl_crc.bit_by_bit(message)`. 

### Communicating with the Bootloader

With that done, we can start communicating with the bootloader. Recall that the bootloader expects:
* To start with `0x00`
* A 16 byte encrypted message (4 bytes signature + 12 bytes data)
* CRC16

We don't really care what the 16 byte message is (just that each is different so that we get a variety of hamming weights), so we'll use the same text/key module from earlier attacks.

We can now run the following block, and we should get `0xA4` back. You may need to run this block a few times to get the right response back.

In [11]:
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import time
message = [0x00]
ktp = AcqKeyTextPattern_Basic(target=target)

# clear serial buffer
num_char = target.ser.inWaiting()
print(target.ser.read(num_char))

key, text = ktp.newPair() #don't care about key here
message.extend(text)

crc = bl_crc.bit_by_bit(text)

message.append(crc >> 8)
message.append(crc & 0xFF)

target.ser.write(message)
time.sleep(0.1)

num_char = target.ser.inWaiting()
response = target.ser.read(num_char)
print("Response: {:02X}".format(ord(response[0])))


Response: A4


### Capturing Traces

With that out of the way, we can proceed to capturing our traces. The normal 5000 traces we capture isn't long enough to get the rounds we care about, so we'll need to increase it (11000 should be fine):

In [ ]:
scope.adc.samples = 11000

We'll be working with Analyzer, so we'll need to use a ChipWhisperer project to store our traces and text:

In [ ]:
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
project = ProjectFormat()
project.setFilename("jupyter_test")
tc = project.getTraceFormat()
ktp = AcqKeyTextPattern_Basic(target=target)

Below you'll find our capture loop. This will be pretty similar to Tutorial B5, but we've added our communication code. We also check the response and just skip the data we get if it isn't correct.

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time
keys = []
N = 100  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    message = [0x00]
    
    num_char = target.ser.inWaiting()
    target.ser.read(num_char)
    
    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)
    
    message.extend(text)
    
    crc = bl_crc.bit_by_bit(text)
    message.append(crc >> 8)
    message.append(crc & 0xFF)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.ser.write(message)
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if ord(response[0]) != 0xA4:
        # Bad response, just skip
        print("Bad response: {:02X}".format(ord(response[0])))
        continue
    
    tc.addTrace(scope.getLastTrace(), text, "", key)
    
tc._isloaded = True
project.traceManager().appendSegment(tc)

With that, we're done with capturing traces! We can now disconnect from the hardware:

In [ ]:
scope.dis()
target.dis()

## Analysis

Now that we have our traces, we can go ahead and perform the attack. As described in the background theory, we'll have to do two attacks - one to get the 14th round key, and another (using the first result) to get the 13th round key. Then, we'll do some post-processing to finally get the 256 bit encryption key.

### 14th Round Key

We can attack the 14th round key with a standard, no-frills CPA attack (using the inverse sbox, since it's a decryption that we're breaking):

In [ ]:
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, InvSBox_output

tm = project.traceManager()

attack = CPA()
leak_model = AES128_8bit(InvSBox_output)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(tm)
attack.setTraceStart(0)
attack.setTracesPerAttack(tm.numTraces())
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

With the setup done, we can actually preform the attack. 11000 samples is a rather large amount to chew through, so if you want a faster attack you can use a smaller range in `attack.setPointRange()`. `(2900, 4200)` will work for XMEGA, while `(1400, 2600)` will work for the STM32F3 (CWLite ARM).

In [ ]:
attack.setPointRange((0, -1))
stats = attack.processTracesNoGUI()

Below you'll find the key that we should recover from this attack. You may want to check what we actually get against this key to make sure the attack is working.

In [ ]:
key = [0xea, 0x79, 0x79, 0x20, 0xc8, 0x71, 0x44, 0x7d, 0x46, 0x62, 0x5f, 0x51, 0x85, 0xc1, 0x3b, 0xcb]
#key = keys[0]

In [ ]:
rec_key = []
for bnum in stats.findMaximums():
    print("Best Guess = 0x{:02X} Corr = {}".format(bnum[0][0], bnum[0][2]))
    rec_key.append(bnum[0][0])

### 13th Round Key

Analyzer doesn't have a leakage model for the 13th round key built in, so we'll need to create our own. An example class is shown below along with the beginning of the setup. **NOTE: You'll need to update `calc_round_key` with the key you found in the last step**

In [ ]:
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, AESLeakageHelper
from chipwhisperer.analyzer.preprocessing.resync_sad import ResyncSAD

class AES256_Round13_Model(AESLeakageHelper):
    def leakage(self, pt, ct, guess, bnum):
        #You must put YOUR recovered 14th round key here - this example may not be accurate!
        calc_round_key = [0xea, 0x79, 0x79, 0x20, 0xc8, 0x71, 0x44, 0x7d, 0x46, 0x62, 0x5f, 0x51, 0x85, 0xc1, 0x3b, 0xcb]
        xored = [calc_round_key[i] ^ pt[i] for i in range(0, 16)]
        block = xored
        block = self.inv_shiftrows(block)
        block = self.inv_subbytes(block)
        block = self.inv_mixcolumns(block)
        block = self.inv_shiftrows(block)
        result = block
        return self.inv_sbox((result[bnum] ^ guess[bnum]))
    
attack = CPA()
leak_model = AES128_8bit(AES256_Round13_Model)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(tm)

#### Resyncing Traces (XMEGA Only)

The traces for the XMEGA version of the firmware become desynced around sample 7000. This is due to a non-constant AES implementation: the code does not always take the same amount of time to run for every input. (It's actually possible to do a timing attack on this AES implementation! We'll stick with our CPA attack for now.)

While this does open up a timing attack, it actually makes our AES attack a little harder, since we'll have to resync the traces. Luckily, this can be done pretty easily by using the ResyncSAD preprocessing module:

In [ ]:
resync_traces = ResyncSAD(tm)
resync_traces.enabled = True
resync_traces.ref_trace = 0
resync_traces.target_window = (9100, 9300)
resync_traces.max_shift = 200
attack.setTraceSource(resync_traces)

#### Running the Attack

Like in the 14th round attack, we can use a smaller range of points to make the attack faster. `(8000,10990)` works well for the XMEGA, while `(6500, 8500)` works well for the STM32F3.

In [ ]:
attack.setTraceStart(0)
attack.setTracesPerAttack(tm.numTraces())
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((0,-1))
stats = attack.processTracesNoGUI()

You can run the block below and the correct key should be printed out:

In [ ]:
rec_key2 = []
for bnum in stats.findMaximums():
    print("Best Guess = 0x{:02X}, Corr = {}".format(bnum[0][0], bnum[0][2]))
    rec_key2.append(bnum[0][0])

This, however, isn't actually the 13th round key. To get the real 13th round key, we'll need to run what we've recovered through a `shiftrows()` and `mixcolumns()` operation:

In [ ]:
from chipwhisperer.analyzer.attacks.models.aes.funcs import shiftrows,mixcolumns
    
real_key2 = shiftrows(rec_key2)
real_key2 = mixcolumns(real_key2)

print("Recovered:", end="")
for subkey in real_key2:
    print(" {:02X}".format(subkey), end="")
print("")

We now have everything we need to recover the full key! We'll start by combining the 13th and 14th round keys:

In [ ]:
rec_key_comb = real_key2.copy()
rec_key_comb.extend(rec_key)

print("Key:", end="")
for subkey in rec_key_comb:
    print(" {:02X}".format(subkey), end="")
print("")

and then we can use the `AES128_8bit` leakage model to recover the first two rounds:

In [ ]:
result = leak_model.keyScheduleRounds(rec_key_comb, 13, 0)
result.extend(leak_model.keyScheduleRounds(rec_key_comb, 13, 1))
print("Key:", end="")
for subkey in result:
    print(" {:02X}".format(subkey), end="")
print("")

You should see a 32 byte key printed out. Open `supersecret.h`, confirm that we have the right key, and celebrate! 

## Recovering the IV

Now that we have the encryption key, we can proceed onto an attack of the next secret value: the IV.

Here, we have the luxury of seeing the source code of the bootloader. This is generally not something we would have access to in the real world, so we'll try not to use it to cheat. (Peeking at `supersecret.h` counts as cheating.) Instead, we'll use the source to help us identify important parts of the power traces.

### Bootloader Source Code

Inside the bootloader's main loop, it does three tasks that we're interested in:

* it decrypts the incoming ciphertext;
* it applies the IV to the decryption's result; and
* it checks for the signature in the resulting plaintext.

This snippet from `bootloader.c` shows all three of the tasks:

```C
// Continue with decryption
trigger_high();                
aes256_decrypt_ecb(&ctx, tmp32);
trigger_low();
             
// Apply IV (first 16 bytes)
for (i = 0; i < 16; i++){
    tmp32[i] ^= iv[i];
}

//Save IV for next time from original ciphertext                
for (i = 0; i < 16; i++){
    iv[i] = tmp32[i+16];
}

// Tell the user that the CRC check was okay
putch(COMM_OK);
putch(COMM_OK);

//Check the signature
if ((tmp32[0] == SIGNATURE1) &&
   (tmp32[1] == SIGNATURE2) &&
   (tmp32[2] == SIGNATURE3) &&
   (tmp32[3] == SIGNATURE4)){
   
   // Delay to emulate a write to flash memory
   _delay_ms(1);
}   
```

This gives us a pretty good idea of how the microcontroller is going to do its job, but if you'd like to go further, you can open the `.lss` file for the binary that was built. This is called a listing file and it lets you see the assembly that the C was compiled and linked to.

### Power Traces

As you can see from both files, after the decryption process, the bootloader executes a few distinct pieces of code:

* To apply the IV, it uses an XOR operation;
* To store the new IV, it copies the previous ciphertext into the IV array;
* It sends two bytes on the serial port;
* It checks the bytes of the signature one by one.

We should be able to recognize these four parts of the code in the power traces. Let's modify our capture routine to find them:

1. We're looking for the original IV, but it's overwritten after each successful decryption. This means we'll have to reset the target before each trace we capture
1. We'd like to skip over all of the decryption process. Recall that the trigger pin is set low after the decryption finishes. This means we can skip over the AES-256 function by triggering on a falling edge instead
1. Depending on the target, we may have to flush the target's serial lines by sending it a bunch of invalid data and looking for a bad CRC return. This slows down the capture process by a lot, so you may want to try without doing this first.
1. We won't need as many samples, so we can reduce how many we capture. 3000 should be sufficient for most targets.

Let's start by reducing our samples and making a function to reset our target (depending on your target, you may need to change the reset pin):

In [15]:
import time
scope.adc.samples = 3000
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

We can trigger on a falling edge by changing `scope.adc.basic_mode` to `"falling_edge"`:

In [13]:
scope.adc.basic_mode = "falling_edge"

We can flush the serial line by sending an invalid message, then checking for a bad CRC return value (`0xA1`). Let's make sure our changes work by getting a trace:

In [57]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
reset_target(scope)
message = [0x00]


num_char = target.ser.inWaiting()
target.ser.read(num_char)

key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

message.extend(text)

crc = bl_crc.bit_by_bit(text)
message.append(crc >> 8)
message.append(crc & 0xFF)

#flush target's serial
okay = 0
while not okay:
    target.ser.write("\0xxxxxxxxxxxxxxxxxx")
    time.sleep(0.005)
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if response:
        if ord(response[0]) == 0xA1:
            okay = 1

scope.arm()

target.ser.write(message)
timeout = 50
# wait for target to finish
while target.isDone() is False and timeout:
    timeout -= 1
    time.sleep(0.01)

try:
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
except IOError as e:
    print('IOError: %s' % str(e))

# run aux stuff that should happen after trace here
num_char = target.ser.inWaiting()
response = target.ser.read(num_char)
if ord(response[0]) != 0xA4:
    # Bad response, just skip
    print("Bad response: {:02X}".format(ord(response[0])))


trace = scope.getLastTrace()


output_notebook()
p = figure()

xrange = range(len(trace))
p.line(xrange, trace, line_color="red")
show(p)

Loading BokehJS ...

You should see 5 different sections:

* 16 XORs
* 16 register loads (this is the new IV being copied over)
* Some serial communication
* The signature check
* The serial line going idle

Different targets have different power traces (for example, on Arm the XORs and register loads are almost identical), but hopefully you can pick out where each section is. For example, on XMEGA:

![XMEGA_Bonus_Trace](https://wiki.newae.com/images/f/f6/Tutorial-A5-Bonus-Trace-Notes.PNG)

With all of these things clearly visible, we have a pretty good idea of how to attack the IV and the signature. We should be able to look at each of the XOR spikes to find each of the IV bytes - each byte is processed on its own. Then, the signature check uses a short-circuiting comparison: as soon as it finds a byte in error, it stops checking the remaining bytes. This type of check is susceptible to a timing attack.

With those things done, we can move onto our capture loop. It's pretty similar to our last one. We're done with Analyzer, so we can store our traces in Python lists (we'll convert to numpy arrays later for easy analysis).

In [80]:
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time
traces = []
keys = []
plaintexts = []
N = 1000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    reset_target(scope)
    message = [0x00]
    

    num_char = target.ser.inWaiting()
    target.ser.read(num_char)
    
    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)
    plaintexts.append(text)
    
    message.extend(text)
    
    crc = bl_crc.bit_by_bit(text)
    message.append(crc >> 8)
    message.append(crc & 0xFF)

    # run aux stuff that should run before the scope arms here
    
    #flush target's serial
    okay = 0
    while not okay:
        target.ser.write("\0xxxxxxxxxxxxxxxxxx")
        time.sleep(0.005)
        num_char = target.ser.inWaiting()
        response = target.ser.read(num_char)
        if response:
            if ord(response[0]) == 0xA1:
                okay = 1
    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.ser.write(message)
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
            continue
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if ord(response[0]) != 0xA4:
        # Bad response, just skip
        print("Bad response: {:02X}".format(ord(response[0])))
        continue
    
    traces.append(scope.getLastTrace())



Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

Capturing traces:   0%|          | 1/1000 [00:00<13:04,  1.27it/s]

Capturing traces:   0%|          | 2/1000 [00:01<11:47,  1.41it/s]

Capturing traces:   0%|          | 3/1000 [00:01<11:03,  1.50it/s]

Capturing traces:   0%|          | 4/1000 [00:02<13:06,  1.27it/s]

Capturing traces:   0%|          | 5/1000 [00:03<13:32,  1.22it/s]

Capturing traces:   1%|          | 6/1000 [00:04<14:05,  1.18it/s]

Capturing traces:   1%|          | 7/1000 [00:05<14:40,  1.13it/s]

Capturing traces:   1%|          | 8/1000 [00:06<15:43,  1.05it/s]

Capturing traces:   1%|          | 9/1000 [00:07<15:25,  1.07it/s]

Capturing traces:   1%|          | 10/1000 [00:08<13:48,  1.20it/s]

Capturing traces:   1%|          | 11/1000 [00:09<12:54,  1.28it/s]

Capturing traces:   1%|          | 12/1000 [00:09<11:33,  1.42it/s]

Capturing traces:   1%|▏         | 13/1000 [00:10<10:49,  1.52it/s]

Capturing traces:   1%|▏         | 14/1000 [00:10<

Capturing traces:  11%|█         | 109/1000 [01:30<14:24,  1.03it/s]

Capturing traces:  11%|█         | 110/1000 [01:31<13:14,  1.12it/s]

Capturing traces:  11%|█         | 111/1000 [01:32<12:54,  1.15it/s]

Capturing traces:  11%|█         | 112/1000 [01:33<12:35,  1.17it/s]

Capturing traces:  11%|█▏        | 113/1000 [01:33<11:58,  1.24it/s]

Capturing traces:  11%|█▏        | 114/1000 [01:34<12:29,  1.18it/s]

Capturing traces:  12%|█▏        | 115/1000 [01:35<13:24,  1.10it/s]

Capturing traces:  12%|█▏        | 116/1000 [01:36<12:00,  1.23it/s]

Capturing traces:  12%|█▏        | 117/1000 [01:37<12:16,  1.20it/s]

Capturing traces:  12%|█▏        | 118/1000 [01:37<11:24,  1.29it/s]

Capturing traces:  12%|█▏        | 119/1000 [01:38<11:48,  1.24it/s]

Capturing traces:  12%|█▏        | 120/1000 [01:39<12:09,  1.21it/s]

Capturing traces:  12%|█▏        | 121/1000 [01:40<13:04,  1.12it/s]

Capturing traces:  12%|█▏        | 122/1000 [01:41<12:41,  1.15it/s]

Capturing traces:  1

Capturing traces:  22%|██▏       | 216/1000 [03:02<09:48,  1.33it/s]

Capturing traces:  22%|██▏       | 217/1000 [03:03<10:21,  1.26it/s]

Capturing traces:  22%|██▏       | 218/1000 [03:03<09:36,  1.36it/s]

Capturing traces:  22%|██▏       | 219/1000 [03:04<09:22,  1.39it/s]

Capturing traces:  22%|██▏       | 220/1000 [03:05<11:02,  1.18it/s]

Capturing traces:  22%|██▏       | 221/1000 [03:06<10:50,  1.20it/s]

Capturing traces:  22%|██▏       | 222/1000 [03:07<09:58,  1.30it/s]

Capturing traces:  22%|██▏       | 223/1000 [03:07<08:56,  1.45it/s]

Capturing traces:  22%|██▏       | 224/1000 [03:08<08:28,  1.53it/s]

Capturing traces:  22%|██▎       | 225/1000 [03:08<08:00,  1.61it/s]

Capturing traces:  23%|██▎       | 226/1000 [03:09<07:42,  1.67it/s]

Capturing traces:  23%|██▎       | 227/1000 [03:10<08:47,  1.47it/s]

Capturing traces:  23%|██▎       | 228/1000 [03:10<09:03,  1.42it/s]

Capturing traces:  23%|██▎       | 229/1000 [03:11<08:22,  1.53it/s]

Capturing traces:  2

Capturing traces:  32%|███▏      | 323/1000 [04:27<08:04,  1.40it/s]

Capturing traces:  32%|███▏      | 324/1000 [04:28<08:27,  1.33it/s]

Capturing traces:  32%|███▎      | 325/1000 [04:29<08:24,  1.34it/s]

Capturing traces:  33%|███▎      | 326/1000 [04:30<08:13,  1.37it/s]

Capturing traces:  33%|███▎      | 327/1000 [04:30<09:02,  1.24it/s]

Capturing traces:  33%|███▎      | 328/1000 [04:31<09:02,  1.24it/s]

Capturing traces:  33%|███▎      | 329/1000 [04:33<10:49,  1.03it/s]

Capturing traces:  33%|███▎      | 330/1000 [04:33<10:23,  1.07it/s]

Capturing traces:  33%|███▎      | 331/1000 [04:34<09:33,  1.17it/s]

Capturing traces:  33%|███▎      | 332/1000 [04:35<09:01,  1.23it/s]

Capturing traces:  33%|███▎      | 333/1000 [04:36<08:27,  1.32it/s]

Capturing traces:  33%|███▎      | 334/1000 [04:36<09:09,  1.21it/s]

Capturing traces:  34%|███▎      | 335/1000 [04:37<09:29,  1.17it/s]

Capturing traces:  34%|███▎      | 336/1000 [04:38<09:39,  1.15it/s]

Capturing traces:  3

Capturing traces:  43%|████▎     | 430/1000 [05:50<07:34,  1.25it/s]

Capturing traces:  43%|████▎     | 431/1000 [05:51<07:13,  1.31it/s]

Capturing traces:  43%|████▎     | 432/1000 [05:51<07:06,  1.33it/s]

Capturing traces:  43%|████▎     | 433/1000 [05:52<07:06,  1.33it/s]

Capturing traces:  43%|████▎     | 434/1000 [05:53<07:44,  1.22it/s]

Capturing traces:  44%|████▎     | 435/1000 [05:54<07:37,  1.23it/s]

Capturing traces:  44%|████▎     | 436/1000 [05:54<06:57,  1.35it/s]

Capturing traces:  44%|████▎     | 437/1000 [05:55<06:46,  1.39it/s]

Capturing traces:  44%|████▍     | 438/1000 [05:56<07:33,  1.24it/s]

Capturing traces:  44%|████▍     | 439/1000 [05:57<07:40,  1.22it/s]

Capturing traces:  44%|████▍     | 440/1000 [05:58<07:06,  1.31it/s]

Capturing traces:  44%|████▍     | 441/1000 [05:58<06:57,  1.34it/s]

Capturing traces:  44%|████▍     | 442/1000 [05:59<07:05,  1.31it/s]

Capturing traces:  44%|████▍     | 443/1000 [06:00<06:55,  1.34it/s]

Capturing traces:  4

Capturing traces:  54%|█████▎    | 537/1000 [07:10<05:15,  1.47it/s]

Capturing traces:  54%|█████▍    | 538/1000 [07:10<04:51,  1.59it/s]

Capturing traces:  54%|█████▍    | 539/1000 [07:11<04:50,  1.58it/s]

Capturing traces:  54%|█████▍    | 540/1000 [07:12<06:17,  1.22it/s]

Capturing traces:  54%|█████▍    | 541/1000 [07:13<06:13,  1.23it/s]

Capturing traces:  54%|█████▍    | 542/1000 [07:14<06:10,  1.23it/s]

Capturing traces:  54%|█████▍    | 543/1000 [07:15<05:50,  1.30it/s]

Capturing traces:  54%|█████▍    | 544/1000 [07:16<06:18,  1.20it/s]

Capturing traces:  55%|█████▍    | 545/1000 [07:17<07:02,  1.08it/s]

Capturing traces:  55%|█████▍    | 546/1000 [07:17<06:32,  1.16it/s]

Capturing traces:  55%|█████▍    | 547/1000 [07:18<06:06,  1.23it/s]

Capturing traces:  55%|█████▍    | 548/1000 [07:19<06:07,  1.23it/s]

Capturing traces:  55%|█████▍    | 549/1000 [07:19<05:41,  1.32it/s]

Capturing traces:  55%|█████▌    | 550/1000 [07:21<06:46,  1.11it/s]

Capturing traces:  5

Capturing traces:  64%|██████▍   | 644/1000 [08:29<03:59,  1.49it/s]

Capturing traces:  64%|██████▍   | 645/1000 [08:30<04:15,  1.39it/s]

Capturing traces:  65%|██████▍   | 646/1000 [08:30<04:02,  1.46it/s]

Capturing traces:  65%|██████▍   | 647/1000 [08:31<04:06,  1.43it/s]

Capturing traces:  65%|██████▍   | 648/1000 [08:31<03:56,  1.49it/s]

Capturing traces:  65%|██████▍   | 649/1000 [08:32<04:09,  1.40it/s]

Capturing traces:  65%|██████▌   | 650/1000 [08:33<03:58,  1.47it/s]

Capturing traces:  65%|██████▌   | 651/1000 [08:34<03:51,  1.51it/s]

Capturing traces:  65%|██████▌   | 652/1000 [08:34<04:06,  1.41it/s]

Capturing traces:  65%|██████▌   | 653/1000 [08:35<03:50,  1.51it/s]

Capturing traces:  65%|██████▌   | 654/1000 [08:35<03:33,  1.62it/s]

Capturing traces:  66%|██████▌   | 655/1000 [08:36<03:26,  1.67it/s]

Capturing traces:  66%|██████▌   | 656/1000 [08:36<03:17,  1.74it/s]

Capturing traces:  66%|██████▌   | 657/1000 [08:37<03:15,  1.75it/s]

Capturing traces:  6

Capturing traces:  75%|███████▌  | 751/1000 [09:44<02:19,  1.78it/s]

Capturing traces:  75%|███████▌  | 752/1000 [09:45<02:19,  1.78it/s]

Capturing traces:  75%|███████▌  | 753/1000 [09:45<02:15,  1.83it/s]

Capturing traces:  75%|███████▌  | 754/1000 [09:46<02:11,  1.88it/s]

Capturing traces:  76%|███████▌  | 755/1000 [09:47<02:33,  1.59it/s]

Capturing traces:  76%|███████▌  | 756/1000 [09:47<02:43,  1.50it/s]

Capturing traces:  76%|███████▌  | 757/1000 [09:48<02:57,  1.37it/s]

Capturing traces:  76%|███████▌  | 758/1000 [09:49<03:06,  1.30it/s]

Capturing traces:  76%|███████▌  | 759/1000 [09:50<02:59,  1.34it/s]

Capturing traces:  76%|███████▌  | 760/1000 [09:51<03:01,  1.33it/s]

Capturing traces:  76%|███████▌  | 761/1000 [09:51<02:43,  1.46it/s]

Capturing traces:  76%|███████▌  | 762/1000 [09:52<02:29,  1.59it/s]

Capturing traces:  76%|███████▋  | 763/1000 [09:53<02:56,  1.34it/s]

Capturing traces:  76%|███████▋  | 764/1000 [09:53<02:54,  1.35it/s]

Capturing traces:  7

Capturing traces:  86%|████████▌ | 858/1000 [11:04<02:13,  1.07it/s]

Capturing traces:  86%|████████▌ | 859/1000 [11:05<02:01,  1.16it/s]

Capturing traces:  86%|████████▌ | 860/1000 [11:06<01:58,  1.18it/s]

Capturing traces:  86%|████████▌ | 861/1000 [11:06<01:47,  1.30it/s]

Capturing traces:  86%|████████▌ | 862/1000 [11:07<01:52,  1.23it/s]

Capturing traces:  86%|████████▋ | 863/1000 [11:08<01:59,  1.15it/s]

Capturing traces:  86%|████████▋ | 864/1000 [11:09<01:44,  1.30it/s]

Capturing traces:  86%|████████▋ | 865/1000 [11:09<01:42,  1.32it/s]

Capturing traces:  87%|████████▋ | 866/1000 [11:10<01:45,  1.27it/s]

Capturing traces:  87%|████████▋ | 867/1000 [11:11<01:39,  1.33it/s]

Capturing traces:  87%|████████▋ | 868/1000 [11:12<01:49,  1.21it/s]

Capturing traces:  87%|████████▋ | 869/1000 [11:13<01:42,  1.28it/s]

Capturing traces:  87%|████████▋ | 870/1000 [11:13<01:39,  1.30it/s]

Capturing traces:  87%|████████▋ | 871/1000 [11:14<01:48,  1.18it/s]

Capturing traces:  8

Capturing traces:  96%|█████████▋| 965/1000 [12:32<00:28,  1.23it/s]

Capturing traces:  97%|█████████▋| 966/1000 [12:33<00:26,  1.28it/s]

Capturing traces:  97%|█████████▋| 967/1000 [12:34<00:28,  1.16it/s]

Capturing traces:  97%|█████████▋| 968/1000 [12:35<00:28,  1.11it/s]

Capturing traces:  97%|█████████▋| 969/1000 [12:36<00:29,  1.06it/s]

Capturing traces:  97%|█████████▋| 970/1000 [12:37<00:30,  1.01s/it]

Capturing traces:  97%|█████████▋| 971/1000 [12:38<00:25,  1.13it/s]

Capturing traces:  97%|█████████▋| 972/1000 [12:39<00:24,  1.16it/s]

Capturing traces:  97%|█████████▋| 973/1000 [12:39<00:21,  1.27it/s]

Capturing traces:  97%|█████████▋| 974/1000 [12:40<00:20,  1.27it/s]

Capturing traces:  98%|█████████▊| 975/1000 [12:41<00:22,  1.14it/s]

Capturing traces:  98%|█████████▊| 976/1000 [12:42<00:23,  1.02it/s]

Capturing traces:  98%|█████████▊| 977/1000 [12:43<00:21,  1.06it/s]

Capturing traces:  98%|█████████▊| 978/1000 [12:44<00:18,  1.19it/s]

Capturing traces:  9

### Analysis

#### Attack Theory

The bootloader applies the IV to the AES decryption result by calculating


$\text{PT} = \text{DR} \oplus \text{IV}$

where DR is the decrypted ciphertext, IV is the secret vector, and PT is the plaintext that the bootloader will use later. We only have access to one of these: since we know the AES-256 key, we can calculate DR. This exclusive or should be visible in the power traces

This is enough information for us to attack a single bit of the IV. Suppose we only wanted to get the first bit (number 0) of the IV. We could do the following:

* Split all of the traces into two groups: those with DR[0] = 0, and those with DR[0] = 1.
* Calculate the average trace for both groups.
* Find the difference between the two averages. It should include a noticeable spike during the first iteration of the loop.
* Look at the direction of the spike to decide if the IV bit is 0 `(PT[0] = DR[0])` or if the IV bit is 1 `(PT[0] = ~DR[0])`.

This is effectively a DPA attack on a single bit of the IV. We can repeat this attack 128 times to recover the entire IV.

#### A 1-Bit Attack

Recall that we're looking for the xor operation between the last decrypted block, so we'll need to decrypt it up to that point. The PyCrypto includes an AES decyprtion routine, so we'll be using that. We'll start by importing the necessary modules and converting our traces/plaintext to numpy arrays:

In [81]:
from Crypto.Cipher import AES
import numpy as np

trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(plaintexts)

numTraces = len(trace_array)
traceLen = len(trace_array[0])

Next we'll do the AES256 decryption. If you got a different key in the earlier part, you'll need to change `knownkey`.

In [82]:
knownkey = [0x94, 0x28, 0x5D, 0x4D, 0x6D, 0xCF, 0xEC, 0x08, 0xD8, 0xAC, 0xDD, 0xF6, 0xBE, 0x25, 0xA4, 0x99,
            0xC4, 0xD9, 0xD0, 0x1E, 0xC3, 0x40, 0x7E, 0xD7, 0xD5, 0x28, 0xD4, 0x09, 0xE9, 0xF0, 0x88, 0xA1]

knownkey = bytes(knownkey)
dr = []
aes = AES.new(knownkey, AES.MODE_ECB)
for i in range(numTraces):
    ct = bytes(textin_array[i])
    pt = aes.decrypt(ct)
    d = [bytearray(pt)[i] for i in range(16)]
    dr.append(d)

Now, let's split the traces into two groups by comparing bit 0 of the DR:

In [83]:
groupedTraces = [[] for _ in range(2)]
for i in range(numTraces):
    bit0 = dr[i][0] & 0x01
    groupedTraces[bit0].append(trace_array[i])
print(len(groupedTraces[0]))

492


If you have 1000 traces, you should expect this to print a number around 500 - roughly half of the traces should fit into each group. Now, NumPy's average function lets us easily calculate the average at each point:

In [84]:
# Find averages and differences
means = []
for i in range(2):
    means.append(np.average(groupedTraces[i], axis=0))
diff = means[1] - means[0]

Finally, we can plot this difference to see if we can spot the IV:

In [85]:
# Split traces into 2 groups
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(diff))
xrange2 = range(len(traces[0]))
p.line(xrange, diff, line_color="red")
#p.line(xrange2, traces[0], line_color='blue')
show(p)

Loading BokehJS ...

You should see a few visible spikes. We're looking for the XOR for byte 0 here, so any later spikes won't be the XOR. Use bokeh's zoom functionality to pinpoint all the largest spikes and record their sample location. You'll probably need to record a few: only one is the correct spike, but we won't be able to tell until we repeat this with other bytes. For example, you might have spikes at 37, 41, and 45. Make sure you record all these values. These peaks won't all be above 0, so make sure you're looking at both positive and negative values.

Next, we'll need to repeat this with a few more bytes. To make things easier, the necessary code has been combined into the below block. Increment the `0` in `bit0 = dr[i][0] & 0x01` to other numbers to attack other bytes. Attacking bytes 0 through 3 should be sufficient.

In [ ]:
groupedTraces = [[] for _ in range(2)]
for i in range(numTraces):
    bit0 = dr[i][0] & 0x01
    groupedTraces[bit0].append(trace_array[i])
print(len(groupedTraces[0]))

# Find averages and differences
means = []
for i in range(2):
    means.append(np.average(groupedTraces[i], axis=0))
diff = means[1] - means[0]

# Split traces into 2 groups
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(diff))
xrange2 = range(len(traces[0]))
p.line(xrange, diff, line_color="red")
show(p)

Now that you have some peak data, you'll want to use this to find the time shift between XORs. This time shift should be constant between samples and needs to work for all samples (each run through the loop is the same, so it makes sense that the time shift should be constant). For example, you might have:

```
0th byte @ 37, 41
1st byte @ 77, 81
2nd byte @ 105, 117, 121
3rd byte @ 141, 157, 161
4th byte @ 197, 201
```

With this data, peaks at 41, 81, 121, 161, and 201 have a constant time shift of 40. This means the location of the XORs is `41 + 40 * byte#`

#### The Other 127

The best way to attack the IV would be to repeat the 1-bit conceptual attack for each of the bits. Try to do this yourself! (Really!) If you're stuck, here are a few hints to get you going:

One easy way of looping through the bits is by using two nested loops, like this:

```python
for byte in range(16):
    for bit in range(8):
        # Attack bit number (byte*8 + bit)
```

The sample that you'll want to look at will depend on which byte you're attacking. We had success when we used `location = 51 + byte*60`, but your mileage will vary.

The bitshift operator and the bitwise-AND operator are useful for getting at a single bit:

```python
# This will either result in a 0 or a 1
checkIfBitSet = (byteToCheck >> bit) & 0x01
```

If you're really, really stuck, the end of this tutorial has a working script. After finding the IV, check `supersecret.h` and verify that your attack was successful.

## Attacking the Signature

Since the bootloader has different execution times based on the number of correct bytes, it's possible to perform a timing attack similar to the one covered in Tutorial B3. If you're feeling very confident in your SPA skills, you may want to attempt this yourself. This attack, however, won't be covered in this tutorial.

The last thing we can do with this bootloader is attack the signature. This final section will show how one byte of the signature could be recovered. If you want more of this kind of analysis, a more complete timing attack is shown in Tutorial B3-1 Timing Analysis with Power for Password Bypass.

### Attack Theory

Recall from earlier that the signature check in C looks like:

```C
if ((tmp32[0] == SIGNATURE1) &&
    (tmp32[1] == SIGNATURE2) &&
    (tmp32[2] == SIGNATURE3) &&
    (tmp32[3] == SIGNATURE4)){
```

Open the listing (`.lss`) file for your binary and find the above check. You should find that the first unsuccessful check will cause it to branch. In C, boolean expressions support short-circuiting. When checking multiple conditions, the program will stop evaluating these booleans as soon as it can tell what the final value will be. In this case, unless all four of the equality checks are true, the result will be false. Thus, as soon as the program finds a single false condition, it's done.

The assembly code confirms this short-circuiting operation. Each of the four assembly blocks include a comparison and a conditional branch. All four of the branches return the program to the same location (the start of the while(1) loop). If any of the comparisons are false, the branches will return the program back to the start of the loop. All four branches must fail to get into the body of the if block.

The short-circuiting conditions are perfect for us. We can use our power traces to watch how long it takes for the signature check to fail. If the check takes longer than usual, then we know that the first byte of our signature was right.

### Finding a Single Byte

Okay, we know that our power trace will look a lot different for one of our choices of signatures. Let's figure out which one. We'll start by finding the average over all of our traces:

In [92]:
# Find the average over all of the traces
mean = np.average(traces, axis=0)
print(mean)

[ 0.03722949 -0.13429297 -0.09866309 ...  0.12121582  0.1380127
  0.13033008]


Then, we'll split our traces into 256 different groups (one for each plaintext). Since we know the IV, we can now use it to recover the actual plaintext that the bootloader checks:

In [93]:
# Split the traces into groups
groupedTraces = [[] for _ in range(256)]
for i in range(numTraces):
    group = dr[i][0] ^ 0xC1
    groupedTraces[group].append(traces[i])

Next, we can find the mean for each group and see how much they differ from the overall mean:

In [97]:
# Find the mean for each group
means = np.zeros([256, traceLen])
for i in range(256):
    if len(groupedTraces[i]) > 0:
        means[i] = np.average(groupedTraces[i], axis=0)
        
p = figure()
colors = ["red", "blue", "green", "yellow"]
xrange = range(len(means[0]))
for i in range(4):
    p.line(xrange, means[i] - mean, line_color=colors[i%4])
    
show(p)

In [91]:
corr = []
for i in range(256):
    print(np.corrcoef(mean[1200:1850], means[i][1200:1850]))
    #corr.append(np.corrcoef(mean[1200:1850], means[i][1200:1850])[0, 1])
print(np.sort(corr))
print(np.argsort(corr))

[[1.         0.99989996]
 [0.99989996 1.        ]]
[[1.         0.99993879]
 [0.99993879 1.        ]]
[[1.         0.99993159]
 [0.99993159 1.        ]]
[[1.         0.99995319]
 [0.99995319 1.        ]]
[[1.        0.9999613]
 [0.9999613 1.       ]]
[[1.         0.99996257]
 [0.99996257 1.        ]]
[[1.         0.99989451]
 [0.99989451 1.        ]]
[[1.         0.99991682]
 [0.99991682 1.        ]]
[[1.         0.99994716]
 [0.99994716 1.        ]]
[[1.         0.99992096]
 [0.99992096 1.        ]]
[[1.         0.99996642]
 [0.99996642 1.        ]]
[[1.         0.99994716]
 [0.99994716 1.        ]]
[[1.         0.99993449]
 [0.99993449 1.        ]]
[[1.         0.99994837]
 [0.99994837 1.        ]]
[[1.         0.99996355]
 [0.99996355 1.        ]]
[[1.         0.99997811]
 [0.99997811 1.        ]]
[[1.         0.99983671]
 [0.99983671 1.        ]]
[[1.        0.9999645]
 [0.9999645 1.       ]]
[[1.        0.9999622]
 [0.9999622 1.       ]]
[[1.         0.99992334]
 [0.99992334 1.   

[[1.         0.99997482]
 [0.99997482 1.        ]]
[[1.         0.99996308]
 [0.99996308 1.        ]]
[[1.         0.99997404]
 [0.99997404 1.        ]]
[[1.         0.99994953]
 [0.99994953 1.        ]]
[[1.         0.99990148]
 [0.99990148 1.        ]]
[[1.         0.99995708]
 [0.99995708 1.        ]]
[[1.         0.99997214]
 [0.99997214 1.        ]]
[[1.         0.99988216]
 [0.99988216 1.        ]]
[[1.         0.99996363]
 [0.99996363 1.        ]]
[[1.         0.99995893]
 [0.99995893 1.        ]]
[[1.         0.99992696]
 [0.99992696 1.        ]]
[[1.         0.99985326]
 [0.99985326 1.        ]]
[[1.         0.99993625]
 [0.99993625 1.        ]]
[[1.         0.99996545]
 [0.99996545 1.        ]]
[[1.         0.99995383]
 [0.99995383 1.        ]]
[[1.         0.99952055]
 [0.99952055 1.        ]]
[[1.         0.99994787]
 [0.99994787 1.        ]]
[[1.         0.99967347]
 [0.99967347 1.        ]]
[[1.         0.99994557]
 [0.99994557 1.        ]]
[[1.         0.99997268]
 [0.99

## Conclusion

We've now successfully recovered the encryption key and the IV for the bootloader!

## Appendix A: IV Attack Script

Make sure you've run the IV analysis blocks before running this block.

In [ ]:
for byte in range(16):
    location = 41 + byte * 40
    iv = 0
    for bit in range(8):
        pt_bits = [((dr[i][byte] >> (7-bit)) & 0x01) for i in range(numTraces)]

        # Split traces into 2 groups
        groupedPoints = [[] for _ in range(2)]
        for i in range(numTraces):
            groupedPoints[pt_bits[i]].append(trace_array[i][location])
            
        means = []
        for i in range(2):
            means.append(np.average(groupedPoints[i]))
        diff = means[1] - means[0]
        
        iv_bit = 1 if diff > 0 else 0
        iv = (iv << 1) | iv_bit
        
        print(iv_bit, end = " ")
        
    print("{:02X}".format(iv))